In [ ]:
import sys, os
import polars as pl
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import viz_tools as viz 

In [ ]:
%load_ext autoreload
%autoreload 2

# Analisis de nuestra economia

In [ ]:
#Carga de datalakes
q_impo = pl.scan_parquet("../Data/datalake_impo/*.parquet")
q_expo = pl.scan_parquet("../Data/datalake_expo/*.parquet")

In [ ]:
#Vemos las columnas en cada uno
print("Columnas en importaciones:")
print(q_impo.collect_schema().names())
print("\nColumnas en exportaciones:")
print(q_expo.collect_schema().names())

In [ ]:

# Diccionario Básico de Capítulos (Para que se entienda el gráfico)
mapa_sectores_arancel = {
    # SECCIÓN I: Animales y sus productos
    "01": "Animales Vivos", "02": "Carnes", "03": "Pescados y Mariscos", "04": "Lácteos y Huevos", "05": "Otros Prod. Animales",
    
    # SECCIÓN II: Vegetales y Frutos (CLAVE COLOMBIA)
    "06": "Plantas y Flores", "07": "Hortalizas", "08": "Frutas y Frutos", "09": "Café, Té y Especias", "10": "Cereales", 
    "11": "Molinería", "12": "Semillas Oleaginosas", "13": "Gomas y Resinas", "14": "Materias Trenzables",
    
    # SECCIÓN III y IV: Grasas y Alimentos Procesados
    "15": "Grasas y Aceites", "16": "Prep. de Carne/Pescado", "17": "Azúcares", "18": "Cacao", "19": "Prep. Cereales/Harina",
    "20": "Prep. Legumbres/Frutas", "21": "Prep. Alimenticias Diversas", "22": "Bebidas y Alcohol", "23": "Alimento Animal", "24": "Tabaco",
    
    # SECCIÓN V: Minerales (EL MOTOR DE EXPORTACIÓN)
    "25": "Sal, Azufre y Tierras", "26": "Minerales Metalíferos", "27": "Combustibles y Petróleo",
    
    # SECCIÓN VI y VII: Química y Plásticos
    "28": "Química Inorgánica", "29": "Química Orgánica", "30": "Farmacéuticos", "31": "Abonos/Fertilizantes", 
    "32": "Extractos Curtientes/Tintes", "33": "Cosméticos/Perfumería", "34": "Jabones y Ceras", "38": "Prod. Químicos Diversos",
    "39": "Plásticos", "40": "Caucho",
    
    # SECCIÓN VIII a X: Pieles, Madera y Papel
    "41": "Pieles y Cueros", "44": "Madera", "47": "Pasta de Madera", "48": "Papel y Cartón", "49": "Libros y Prensa",
    
    # SECCIÓN XI: Textiles y Confecciones
    "50": "Seda", "51": "Lana", "52": "Algodón", "61": "Prendas de Vestir (Punto)", "62": "Prendas de Vestir (No Punto)", "63": "Otros Textiles",
    
    # SECCIÓN XV a XVII: Metales, Maquinaria y Vehículos (CLAVE IMPORTACIÓN)
    "72": "Fundición de Hierro/Acero", "73": "Manufacturas de Hierro/Acero", "74": "Cobre", "76": "Aluminio",
    "84": "Maquinaria y Reactores", "85": "Aparatos Eléctricos y Sonido", "87": "Vehículos Automóviles", "88": "Aeronaves",
    
    # SECCIÓN XVIII a XXI: Varios
    "90": "Instrumentos Ópticos/Médicos", "94": "Muebles y Luminarias", "95": "Juguetes y Deportes", "96": "Manufacturas Diversas"
}
# Analicemos EXPORTACIONES (¿Qué vende Colombia?)
df_productos_exportaciones = (
    q_expo # Usamos el LazyFrame de Expo
    .with_columns([
        # Extraemos los 2 primeros dígitos de la Posición Arancelaria (POSAR) y el año
        pl.col("POSAR").cast(pl.String).str.slice(0, 2).alias("CAPITULO_COD"),
        pl.col("ANIO").alias("ANIO")
    ])
    .group_by("CAPITULO_COD", "ANIO")
    .agg(pl.col("FOBDOL").sum().alias("VALOR_TOTAL"))
    #quitemos el año de la pandemia para ver una tendencia más clara
    #.filter(pl.col("ANIO") != "2020")
    .sort("VALOR_TOTAL", descending=True)
    .limit(15) # Top 15 sectores
    .collect()
)
df_productos_importaciones = (
    q_impo # Usamos el LazyFrame de Impo
    .with_columns([
        # Extraemos los 2 primeros dígitos de la Posición Arancelaria (NABAN)
        pl.col("NABAN").cast(pl.String).str.slice(0, 2).alias("CAPITULO_COD"),
        pl.col("ANIO").alias("ANIO")
    ])
    .group_by("CAPITULO_COD", "ANIO")
    .agg(pl.col("VAFODO").sum().alias("VALOR_TOTAL"))
    #quitemos el año de la pandemia para ver una tendencia más clara
    #.filter(pl.col("ANIO") != "2020")
    .sort("VALOR_TOTAL", descending=True)
    .limit(15) # Top 15 sectores
    .collect()
)
# Mapeamos nombres
df_productos_exportaciones = df_productos_exportaciones.with_columns(
    pl.col("CAPITULO_COD").replace_strict(mapa_sectores_arancel, default="Otros").alias("SECTOR")
)
df_productos_importaciones = df_productos_importaciones.with_columns(
    pl.col("CAPITULO_COD").replace_strict(mapa_sectores_arancel, default="Otros").alias("SECTOR")
)
# Visualización: Treemap (Ideal para ver participaciones de mercado)
datos_treemap_expo = df_productos_exportaciones.to_pandas()
datos_treemap_impo = df_productos_importaciones.to_pandas()

fig_expo = viz.graficar_treemap_productos(
    df_productos_exportaciones,
    path_cols=["SECTOR"],
    value_col="VALOR_TOTAL",
    titulo="Principales Sectores de Exportación de Colombia",
    )
fig_expo.show()
fig_impo = viz.graficar_treemap_productos(
    df_productos_importaciones,
    path_cols=["SECTOR"],
    value_col="VALOR_TOTAL",
    titulo="Principales Sectores de Importación de Colombia",
    )
fig_impo.show()

In [ ]:
%reload_ext autoreload

In [ ]:
# Análisis de Balanza Comercial (Importaciones vs Exportaciones)
df_impo_prep = (
    q_impo
    .with_columns([
        pl.col("ANIO").alias("ANIO"),
        pl.lit("Importación").alias("TIPO"),
        pl.col("VAFODO").alias("VALOR") # Renombrar a genérico
    ])
    .select(["ANIO", "TIPO", "VALOR"])
)

df_expo_prep = (
    q_expo
    .with_columns([
        pl.col("ANIO").alias("ANIO"),
        pl.lit("Exportación").alias("TIPO"),
        pl.col("FOBDOL").alias("VALOR") # Renombrar a genérico
    ])
    .select(["ANIO", "TIPO", "VALOR"])
)

# 3. Unir (Concatenar) Verticalmente
df_balanza = pl.concat([df_impo_prep, df_expo_prep])

# 4. Agrupar para el Gráfico
datos_grafico = (
    df_balanza
    .group_by(["ANIO", "TIPO"])
    .agg(pl.col("VALOR").sum())
    .sort("ANIO")
    .collect()
    .to_pandas()
)

# 5. Graficar usando tu función
fig = viz.graficar_balanza_comercial(datos_grafico)
fig.show()

# 6. Calcular Déficit/Superávit (Opcional: Matemática financiera)
df_pivot = datos_grafico.pivot(index='ANIO', columns='TIPO', values='VALOR')
df_pivot['Balanza'] = df_pivot['Exportación'] - df_pivot['Importación']
print("Balanza Comercial (USD):")
print(df_pivot)

In [ ]:
mapa_dian_impo_nombres = {
    "013": "AFGANISTAN", "017": "ALBANIA", "023": "ALEMANIA", "026": "ARMENIA",
    "027": "ARUBA", "029": "BOSNIA-HERZEGOVINA", "031": "BURKINA FASSO", "037": "ANDORRA",
    "040": "ANGOLA", "041": "ANGUILLA", "043": "ANTIGUA Y BARBUDA", "047": "ANTILLAS HOLANDESAS",
    "053": "ARABIA SAUDITA", "059": "ARGELIA", "063": "ARGENTINA", "069": "AUSTRALIA",
    "072": "AUSTRIA", "074": "AZERBAIJAN", "077": "BAHAMAS", "080": "BAHREIN",
    "081": "BANGLADESH", "083": "BARBADOS", "087": "BELGICA", "088": "BELICE",
    "090": "BERMUDAS", "091": "BELARUS", "093": "BIRMANIA (MYANMAR)", "097": "BOLIVIA",
    "101": "BOTSWANA", "105": "BRASIL", "108": "BRUNEI DARUSSALAM", "111": "BULGARIA",
    "115": "BURUNDI", "119": "BUTAN", "127": "CABO VERDE", "137": "CAIMAN, ISLAS",
    "141": "CAMBOYA (KAMPUCHEA)", "145": "CAMERUN, REPUBLICA UNIDA DEL", "149": "CANADA", "159": "SANTA SEDE",
    "165": "COCOS (KEELING), ISLAS", "169": "COLOMBIA", "173": "COMORAS", "177": "CONGO",
    "183": "COOK, ISLAS", "187": "COREA (NORTE), REPUBLICA POPULAR DEMOCRATICA DE", 
    "190": "COREA (SUR), REPUBLICA DE", "193": "COSTA DE MARFIL", "196": "COSTA RICA", "198": "CROACIA",
    "199": "CUBA", "203": "CHAD", "211": "CHILE", "215": "CHINA", "218": "TAIWAN (FORMOSA)",
    "221": "CHIPRE", "229": "BENIN", "232": "DINAMARCA", "235": "DOMINICA", "239": "ECUADOR",
    "240": "EGIPTO", "242": "EL SALVADOR", "243": "ERITREA", "244": "EMIRATOS ARABES UNIDOS",
    "245": "ESPAÑA", "246": "ESLOVAQUIA", "247": "ESLOVENIA", "249": "ESTADOS UNIDOS",
    "251": "ESTONIA", "253": "ETIOPIA", "259": "FEROE, ISLAS", "267": "FILIPINAS",
    "271": "FINLANDIA", "275": "FRANCIA", "281": "GABON", "285": "GAMBIA", "287": "GEORGIA",
    "289": "GHANA", "293": "GIBRALTAR", "297": "GRANADA", "301": "GRECIA", "305": "GROENLANDIA",
    "309": "GUADALUPE", "313": "GUAM", "317": "GUATEMALA", "325": "GUAYANA FRANCESA",
    "329": "GUINEA", "331": "GUINEA ECUATORIAL", "334": "GUINEA-BISSAU", "337": "GUYANA",
    "341": "HAITI", "345": "HONDURAS", "351": "HONG KONG", "355": "HUNGRIA", "361": "INDIA",
    "365": "INDONESIA", "369": "IRAK", "372": "IRAN, REPUBLICA ISLAMICA DEL", "375": "IRLANDA (EIRE)",
    "379": "ISLANDIA", "383": "ISRAEL", "386": "ITALIA", "391": "JAMAICA", "399": "JAPON",
    "403": "JORDANIA", "406": "KAZAJSTAN", "410": "KENIA", "411": "KIRIBATI",
    "412": "KIRGUIZISTAN", "413": "KUWAIT", "420": "LAOS, REPUBLICA POPULAR DEMOCRATICA DE", 
    "426": "LESOTHO", "429": "LETONIA", "431": "LIBANO", "434": "LIBERIA",
    "438": "LIBIA (INCLUYE FEZZAN)", "440": "LIECHTENSTEIN", "443": "LITUANIA", "445": "LUXEMBURGO",
    "447": "MACAO", "448": "MACEDONIA", "450": "MADAGASCAR", "455": "MALAYSIA",
    "458": "MALAWI", "461": "MALDIVAS", "464": "MALI", "467": "MALTA",
    "469": "MARIANAS DEL NORTE, ISLAS", "472": "MARSHALL, ISLAS", "474": "MARRUECOS", "477": "MARTINICA",
    "485": "MAURICIO", "488": "MAURITANIA", "493": "MEXICO", "494": "MICRONESIA, ESTADOS FEDERADOS DE",
    "496": "MOLDAVIA", "497": "MONGOLIA", "498": "MONACO", "501": "MONSERRAT, ISLA",
    "505": "MOZAMBIQUE", "507": "NAMIBIA", "508": "NAURU", "511": "NAVIDAD (CHRISTMAS), ISLAS",
    "517": "NEPAL", "521": "NICARAGUA", "525": "NIGER", "528": "NIGERIA", "531": "NIUE, ISLA",
    "535": "NORFOLK, ISLA", "538": "NORUEGA", "542": "NUEVA CALEDONIA", "545": "PAPUASIA NUEVA GUINEA",
    "548": "NUEVA ZELANDIA", "551": "VANUATU", "556": "OMAN", "566": "PACIFICO, ISLAS (USA)",
    "573": "PAISES BAJOS (HOLANDA)", "576": "PAKISTAN", "578": "PALAU, ISLAS", "580": "PANAMA",
    "586": "PARAGUAY", "589": "PERU", "593": "PITCAIRN, ISLA", "599": "POLINESIA FRANCESA",
    "603": "POLONIA", "607": "PORTUGAL", "611": "PUERTO RICO", "618": "QATAR",
    "628": "REINO UNIDO", "640": "REPUBLICA CENTROAFRICANA", "644": "REPUBLICA CHECA", 
    "647": "REPUBLICA DOMINICANA", "660": "REUNION", "665": "ZIMBABWE", "670": "RUMANIA",
    "675": "RUANDA", "676": "RUSIA", "677": "SALOMON, ISLAS", "685": "SAHARA OCCIDENTAL",
    "687": "SAMOA", "690": "SAMOA NORTEAMERICANA", "695": "SAN CRISTOBAL Y NIEVES", 
    "697": "SAN MARINO", "700": "SAN PEDRO Y MIGUELON", "705": "SAN VICENTE Y LAS GRANADINAS", 
    "710": "SANTA ELENA", "715": "SANTA LUCIA", "720": "SANTO TOME Y PRINCIPE", "728": "SENEGAL",
    "729": "SERBIA", "731": "SEYCHELLES", "735": "SIERRA LEONA", "741": "SINGAPUR",
    "744": "SIRIA, REPUBLICA ARABE DE", "748": "SOMALIA", "750": "SRI LANKA", 
    "756": "SUDAFRICA, REPUBLICA DE", "759": "SUDAN", "764": "SUECIA", "767": "SUIZA",
    "770": "SURINAM", "773": "SWAZILANDIA", "774": "TADJIKISTAN", "776": "TAILANDIA",
    "780": "TANZANIA, REPUBLICA UNIDA DE", "783": "DJIBOUTI", 
    "787": "TERRITORIO BRITANICO DEL OCEANO INDICO", "788": "TIMOR DEL ESTE", "800": "TOGO",
    "805": "TOKELAU", "810": "TONGA", "815": "TRINIDAD Y TOBAGO", "820": "TUNICIA",
    "823": "TURCAS Y CAICOS, ISLAS", "825": "TURKMENISTAN", "827": "TURQUIA", "828": "TUVALU",
    "830": "UCRANIA", "833": "UGANDA", "845": "URUGUAY", "847": "UZBEKISTAN",
    "850": "VENEZUELA", "855": "VIETNAM", "863": "VIRGENES, ISLAS (BRITANICAS)", 
    "866": "VIRGENES, ISLAS (NORTEAMERICANAS)", "870": "FIJI", "875": "WALLIS Y FORTUNA, ISLAS",
    "880": "YEMEN", "885": "YUGOSLAVIA", "888": "ZAIRE", "890": "ZAMBIA",
    "897": "ZONA NEUTRAL PALESTINA", "911": "ZONA FRANCA DE BARRANQUILLA", 
    "912": "ZONA FRANCA DE BUENAVENTURA", "913": "ZONA FRANCA DE PALMASECA - CALI", 
    "914": "ZONA FRANCA DE CUCUTA", "915": "ZONA FRANCA DE SANTA MARTA", 
    "916": "ZONA FRANCA DE CARTAGENA", "917": "ZONA FRANCA DE RIONEGRO MEDELLIN", 
    "918": "ZONA FRANCA DE CANDELARIA - CARTAGENA", "919": "ZONA FRANCA DE BOGOTA", 
    "920": "ZONA FRANCA DE PACIFICO - CALI", "921": "ZONA FRANCA DE BARU BEACH RESORT", 
    "922": "ZONA FRANCA DE POZOS COLORADOS", "923": "ZONA FRANCA DE EUROCARIBE DE INDIAS", 
    "924": "ZONA FRANCA DEL EJE CAFETERO", "928": "ZFP La Cayena", "999": "NO DECLARADOS"
}
mapa_dian_iso = {
    "249": "USA", # Estados Unidos
    "215": "CHN", # China
    "493": "MEX", # México
    "072": "BRA", # Brasil
    "105": "DEU", # Alemania
    "169": "COL", # Colombia (Reimportaciones)
    "361": "IND", # India
    "239": "ECU", # Ecuador
    "275": "FRA", # Francia
    "399": "JPN", # Japón
    "063": "ARG", # Argentina
    "410": "KOR", # Corea del Sur
    "385": "ITA", # Italia
    "245": "ESP", # España
    "589": "PER", # Perú
    "149": "CAN", # Canadá
    "521": "NLD", # Países Bajos (Holanda)
    "827": "TUR", # Turquía
    "850": "VEN", # Venezuela
    "196": "CRI", # Costa Rica
    "152": "CHL", # Chile
    "767": "CHE", # Suiza
    "809": "SWE", # Suecia
    "628": "GBR", # Reino Unido
    "069": "AUS", # Australia
    "607": "PRT", # Portugal
    "059": "BEL", # Bélgica
    "880": "VNM", # Vietnam
    "379": "IDN", # Indonesia
    "744": "SGP", # Singapur
    "764": "THA", # Tailandia
    "450": "MYS", # Malasia
    "603": "POL", # Polonia
    "687": "RUS", # Rusia
    "813": "TWN", # Taiwán
    "375": "HKG", # Hong Kong
    "240": "EGY", # Egipto
    "576": "PAK", # Pakistán
    "244": "ARE", # Emiratos Árabes Unidos
    "391": "ISR", # Israel
    "053": "SAU",  # Arabia Saudita
    "211": "AUT",   # Austria
    "386": "DNK",  # Dinamarca
    "428": "NOR",  # Noruega
    "498": "FIN",  # Finlandia
    "855": "IRL",  # Irlanda
    "023": "ALB",  # Albania
    "218": "TZA",  # Tanzania
    "156": "UGA",  # Uganda
    "190": "KAZ",  # Kazajistán
    "840": "QAT",  # Qatar

}

In [ ]:
# 1. Limpiamos q_impo
q_impo_clean = (
    q_impo
    .rename({
        'PAISGEN': 'PAIS',      # Unificamos nombre
        'VAFODO': 'VALOR_USD',  # Unificamos nombre
        'NABAN': 'CODIGO'       # Unificamos nombre
    })
    .select(['ANIO', 'PAIS', 'VALOR_USD', 'CODIGO']) # Solo lo que sirve
    .with_columns([
        pl.lit("IMPO").alias("TIPO"),       # Etiqueta para diferenciar
        pl.col("VALOR_USD").cast(pl.Float64) # Asegurar que sea número
    ])
)
#Mapear nombres de países
q_impo_clean = q_impo_clean.with_columns(
    pl.col("PAIS").replace_strict(mapa_dian_iso,
        default=pl.col("PAIS"))
)
# 2. Limpiamos q_expo
q_expo_clean = (
    q_expo
    .rename({
        # 'PAIS' ya suele venir bien en expo, si no, renómbralo aquí
        'COD_PAI4': 'PAIS',     # Unificamos nombre
        'FOBDOL': 'VALOR_USD',
        'POSAR': 'CODIGO'
    })
    .select([ 'ANIO', 'PAIS', 'VALOR_USD', 'CODIGO'])
    .with_columns([
        pl.lit("EXPO").alias("TIPO"),
        pl.col("VALOR_USD").cast(pl.Float64)
    ])
)
#Mapear nombres de países
q_expo_clean = q_expo_clean.with_columns(
    pl.col("CODIGO").replace_strict(mapa_dian_iso,
        default=pl.col("CODIGO"))
)

# 3. Creamos el LazyFrame Maestro (Aún no procesa nada)
q_total = pl.concat([q_impo_clean, q_expo_clean])
print("Nombres de columnas:", q_total.collect_schema().names())

In [ ]:
df_balanza = (
    q_total
    .group_by("PAIS")
    .agg([
        pl.col("VALOR_USD").filter(pl.col("TIPO") == "EXPO").sum().fill_null(0).alias("TOTAL_EXPO"),
        pl.col("VALOR_USD").filter(pl.col("TIPO") == "IMPO").sum().fill_null(0).alias("TOTAL_IMPO")
    ])
    .with_columns(
        (pl.col("TOTAL_EXPO") - pl.col("TOTAL_IMPO")).alias("SALDO")
    )
    .collect()
    .with_columns(
        pl.col("PAIS"))
    .to_pandas()
)
print(df_balanza[df_balanza["PAIS"] == "USA"])

In [ ]:
fig = viz.graficar_saldo_paises(df_balanza, top_n=30
                                )
fig.show()

In [ ]:
df_mapa = (
    q_expo_clean
    .group_by("PAIS")
    .agg(pl.col("VALOR_USD").sum().alias("VALOR_TOTAL"))
    .collect()
)

# 2. Aplicar el mapeo

df_mapa = df_mapa.with_columns(
    pl.col("PAIS")
    .cast(pl.String)             # Asegurar que sea texto
    .str.replace(".0", "", literal=True) # Quitar decimales si quedaron (ej "249.0" -> "249")
    .replace(mapa_dian_impo_nombres)      # <--- AQUÍ OCURRE LA MAGIA
    .alias("ISO_CODE")           # Nueva columna para Plotly
)

# 3. Filtrar los que no encontraron mapeo (quedan igual al código original)
# Para que el mapa no falle, nos quedamos solo con los que tienen formato ISO (3 letras)
# Opcional: imprimir los que faltaron para agregarlos al diccionario luego
faltantes = df_mapa.filter(pl.col("ISO_CODE").str.len_chars() != 3)
if not faltantes.is_empty():
    print(f"Advertencia: {len(faltantes)} códigos de país no se encontraron en el diccionario.")
    print(faltantes.head(5))

# 4. Convertir a Pandas para Plotly
pdf_mapa = df_mapa.to_pandas()

# 5. Graficar
fig_map = viz.ma(
    pdf_mapa,
    locations="ISO_CODE",     # Usamos la columna mapeada
    color="VALOR_TOTAL",
    hover_name="PAIS",     # Muestra el código DIAN al pasar el mouse
    
    title="Mapa de EXPO (Origen)",
    labels={"VALOR_TOTAL": "Valor FOB (USD)"}
)

# Mejorar proyección (que no se vea plano)
fig_map.update_geos(projection_type="natural earth")
fig_map.show()

In [ ]:
def calcular_variacion_porcentual(lf_polars, tipo_flujo="Exportación", umbral_min_usd=100000):
    """
    Calcula la variación porcentual anual por sector.
    umbral_min_usd: Filtra sectores con muy poco volumen para evitar % infinitos o ruidosos.
    """
    # 1. Agrupación y cálculo (Trabajando en Lazy)
    lf_calculo = (
        lf_polars
        .filter(pl.col("TIPO") == tipo_flujo)
        .group_by(["ANIO", "CODIGO"])
        .agg(pl.col("VALOR_USD").sum().alias("TOTAL_ANUAL"))
        .sort(["CODIGO", "ANIO"])
        .with_columns(
            VALOR_ANTERIOR = pl.col("TOTAL_ANUAL").shift(1).over("CODIGO")
        )
        .with_columns(
            # Fórmula: ((Actual - Anterior) / Anterior) * 100
            VARIACION_PCT = ((pl.col("TOTAL_ANUAL") - pl.col("VALOR_ANTERIOR")) / pl.col("VALOR_ANTERIOR")) * 100
        )
        .filter(
            (pl.col("VARIACION_PCT").is_not_null()) & 
            (pl.col("VALOR_ANTERIOR") > umbral_min_usd) # Filtro de relevancia
        )
    )

    # 2. Ejecutar el plan (Collect)
    df_final = lf_calculo.collect()

    # 3. Obtener el último año de forma segura
    ultimo_anio = df_final.select(pl.col("ANIO")).max().item()
    
    return df_final.filter(pl.col("ANIO") == ultimo_anio).to_pandas()

In [ ]:
# 1. Analizar Exportaciones
df_var_expo = calcular_variacion_porcentual(q_total, "Exportación")
fig_expo = viz.graficar_variacion_pct(df_var_expo, "Exportación")
fig_expo.show()

# 2. Analizar Importaciones
df_var_impo = calcular_variacion_porcentual(q_total, "Importación")
fig_impo = viz.graficar_variacion_pct(df_var_impo, "Importación")
fig_impo.show()

In [ ]:
df_var_pct_expo = calcular_variacion_porcentual(q_total, tipo_flujo="Exportación")

# 3. Llamamos a la función de visualización de tu script src/viz_tools.py

fig_pct = viz.graficar_variacion_pct(df_var_pct_expo, tipo_flujo="Exportación")

# 4. Mostrar
fig_pct.show()